In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Trading Strategy & Backtest — Stock Market Analytics\n",
    "\n",
    "Notebook para experimentar geração de sinais e simulação de estratégias.\n",
    "\n",
    "> Nota: a versão “oficial” é feita via scripts em `src/strategy/` e `src/backtest/`,\n",
    "mas aqui você pode iterar parâmetros e visualizar rapidamente."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 0. Setup"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from pathlib import Path\n",
    "\n",
    "from src import config\n",
    "from src.utils.io import load_parquet, save_parquet\n",
    "from src.backtest.simulator import backtest, Costs, compute_metrics\n",
    "\n",
    "IMG_DIR = config.IMG_DIR\n",
    "IMG_DIR.mkdir(parents=True, exist_ok=True)\n",
    "\n",
    "def savefig(path: Path):\n",
    "    path.parent.mkdir(parents=True, exist_ok=True)\n",
    "    plt.tight_layout()\n",
    "    plt.savefig(path, dpi=120)\n",
    "    print(\"Figura salva em\", path)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Carregar preços e sinais\n",
    "- `data/processed/prices.parquet`\n",
    "- `data/signals/signals.parquet` (gerado por `src/strategy/generate_signals.py`)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "prices = load_parquet(config.PROCESSED_DIR / \"prices.parquet\")\n",
    "signals = load_parquet(config.SIGNALS_DIR / \"signals.parquet\")\n",
    "\n",
    "prices = prices[[\"date\", \"ticker\", \"adjclose\"]].copy()\n",
    "signals = signals[[\"date\", \"ticker\", \"signal_cls\", \"signal_reg\"]].copy()\n",
    "\n",
    "prices.head(), signals.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Selecionar coluna de sinal e construir posições (execução next-day)\n",
    "Você pode alternar entre `signal_cls` e `signal_reg`."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "signal_col = \"signal_cls\"  # altere para \"signal_reg\" se preferir\n",
    "sig = signals[[\"date\", \"ticker\", signal_col]].rename(columns={signal_col: \"signal\"}).copy()\n",
    "sig[\"date\"] = pd.to_datetime(sig[\"date\"]).dt.tz_localize(None)\n",
    "sig = sig.sort_values([\"ticker\", \"date\"]) \n",
    "\n",
    "positions = sig.copy()\n",
    "positions[\"position\"] = positions.groupby(\"ticker\")[\"signal\"].shift(1).fillna(0).astype(int)\n",
    "positions = positions[[\"date\", \"ticker\", \"position\"]]\n",
    "\n",
    "positions.head(10)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Rodar backtest (carteira equally-weighted)\n",
    "Parâmetros de custo: `fee_bps` e `slippage_bps` em **basis points** (1 bps = 0.01%)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "costs = Costs(fee_bps=5.0, slippage_bps=2.0)\n",
    "positions_out, equity = backtest(prices, positions, costs)\n",
    "equity.tail()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Métricas & Curvas\n",
    "- CAGR, Sharpe, Vol anualizada, Max Drawdown\n",
    "- Equity curve e Drawdown plot (salvos em `reports/img/`)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "summary = compute_metrics(\n",
    "    equity=equity.set_index(\"date\")[\"equity\"],\n",
    "    ret_daily=equity.set_index(\"date\")[\"ret_port\"],\n",
    ")\n",
    "summary"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Equity curve\n",
    "plt.figure(figsize=(10, 4))\n",
    "plt.plot(equity[\"date\"], equity[\"equity\"])  # cor padrão\n",
    "plt.title(f\"Equity Curve — {signal_col}\")\n",
    "plt.xlabel(\"Date\")\n",
    "plt.ylabel(\"Equity (base=1.0)\")\n",
    "savefig(IMG_DIR / \"equity_curve.png\")\n",
    "plt.close()\n",
    "\n",
    "# Drawdown\n",
    "eq = equity.set_index(\"date\")[\"equity\"]\n",
    "roll_max = eq.cummax()\n",
    "dd = eq / roll_max - 1.0\n",
    "\n",
    "plt.figure(figsize=(10, 2.8))\n",
    "plt.plot(dd.index, dd.values)\n",
    "plt.title(\"Drawdown\")\n",
    "plt.xlabel(\"Date\")\n",
    "plt.ylabel(\"Drawdown\")\n",
    "savefig(IMG_DIR / \"drawdown.png\")\n",
    "plt.close()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Salvar resultados (opcional)\n",
    "Você pode persistir os resultados para conferência posterior."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "save_parquet(positions_out, config.BACKTESTS_DIR / \"positions_notebook.parquet\", index=False)\n",
    "save_parquet(equity, config.BACKTESTS_DIR / \"equity_notebook.parquet\", index=False)\n",
    "summary"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.11"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
